In [ ]:
!pip install xarray
!pip install ipytree

In [1]:
from dask.distributed import Client, progress, LocalCluster
from rechunker import rechunk
import s3fs
#import xarray as xr
import zarr
import dask.array as dsa
import shutil
from dask.diagnostics import ProgressBar

## Start the Dask Cluster

In [2]:
cluster = LocalCluster(n_workers=20)
client = Client(cluster)
client
# client.close()
# cluster.close()

Client Scheduler: tcp://127.0.0.1:35341 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 20 Cores: 80 Memory: 524.29 GB


## Load the MUR SST Dataset from AWS PDS S3

In [3]:
s3 = s3fs.S3FileSystem(anon=True, client_kwargs=dict(region_name='us-west-2'))
s3_store = s3fs.S3Map(root='mur-sst/zarr', s3=s3, check=False)
ds_zarr = zarr.open_consolidated(s3_store, mode='r')

In [ ]:
%%time
#ds = xr.open_zarr(s3_store, consolidated=True)

In [ ]:
print(zarr.tree(ds_zarr))

## Rechunker Plan

In [4]:
s3 = s3fs.S3FileSystem(client_kwargs=dict(region_name='us-west-2'), default_fill_cache=False, skip_instance_cache=True)
s3_rechunk_store = s3fs.S3Map(root='ds-projects/eodc/mursst/6443x100x100-group.zarr', create=True, s3=s3)
# Note this path must exist in S3 or will raise rechunker assertion, `assert temp_store_or_group is not None`
s3_tmp_store = s3fs.S3Map(root='ds-projects/eodc/mursst/tmp.zarr', create=True, s3=s3)

In [6]:
target_chunks = {
    'analysed_sst': {'time': 6443, 'lat': 100, 'lon': 100},
    'analysis_error': {'time': 6443, 'lat': 100, 'lon': 100},
    'mask': {'time': 6443, 'lat': 100, 'lon': 100},
    'sea_ice_fraction': {'time': 6443, 'lat': 100, 'lon': 100},
    'lat': None,
    'lon': None,
    'time': None
}
max_mem = '2GB'

array_plan = rechunk(ds_zarr, target_chunks, max_mem, s3_rechunk_store, s3_tmp_store)
array_plan

{}
hi
<zarr.hierarchy.Group '/'>
{}
hi
<zarr.hierarchy.Group '/'>
{}
hi
<zarr.hierarchy.Group '/'>
{}
hi
<zarr.hierarchy.Group '/'>
{}
{}
{}


<Rechunked>
* Source      : <zarr.hierarchy.Group '/' read-only>
* Intermediate: <zarr.hierarchy.Group '/'>
* Target      : <zarr.hierarchy.Group '/'>

In [ ]:
with ProgressBar():
    array_plan.execute()